In [505]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import colors
import gif

In [ ]:
def get_neighbours(grid_size: np.int64) -> npt.NDArray[np.int16]:
    L: np.int16 = int(np.sqrt(grid_size))
    neighbours = np.zeros((grid_size, 4), dtype=np.int16)
    for k in np.arange(grid_size):
        neighbours[k, 0] = k + 1
        if (k + 1) % L == 0:
            neighbours[k, 0] = k + 1 - L
        neighbours[k, 1] = k + L
        if k > (grid_size - L - 1):
            neighbours[k, 1] = k + L - grid_size
        neighbours[k, 2] = k - 1
        if k % L == 0:
            neighbours[k, 2] = k + L - 1
        neighbours[k, 3] = k - L
        if k < L:
            neighbours[k, 3] = k + grid_size - L
    return neighbours

In [ ]:
# constants

SUSCEPTIBLE = 0
CONTAMINATED = 1
RECOVERED = 2

COLORS: list[str] = ["blue", "red", "green"]
HEALTH_CONDITIONS_IDS: list[str] = ["Susceptible", "Contaminated", "Recovered"]


In [ ]:
conditions: list[dict] = [
    {"prob_contamination": 0.9, "prob_recovery": 0.1},
    {"prob_contamination": 0.5, "prob_recovery": 0.5},
    {"prob_contamination": 0.1, "prob_recovery": 0.9}
]

In [ ]:
# visualization functions

@gif.frame
def draw_grid(grid: npt.NDArray[np.int8]) -> None:
    # create discrete colormap
    cmap = colors.ListedColormap(["blue", "red", "green"])
    bounds = [0, 1, 2]

    grid = np.reshape(grid, newshape=(int(np.sqrt(grid.size)), int(np.sqrt(grid.size))))

    plt.figure(figsize=(8,4))
    plt.title(f"{grid.shape} grid")
    plt.xticks(ticks=np.arange(grid.shape[0] + 1))
    plt.yticks(ticks=np.arange(grid.shape[0] + 1))
    patches = [mpatches.Patch(color=COLORS[i], label=f"{HEALTH_CONDITIONS_IDS[i]}") for i in np.arange(3)]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
    plt.grid(visible=True, which="both")
    plt.imshow(grid, cmap=cmap, origin="lower", extent=(0, grid.shape[0], 0, grid.shape[0]))

def create_gif(frames: npt.NDArray[np.int8]) -> None:
    gif.save(frames, "example.gif", duration=200)

In [ ]:
def spread(val: np.int8, neighbours_health: npt.NDArray[np.int16], conditions: dict) -> np.int8:
    prob_contamination: np.float32 = conditions["prob_contamination"]
    prob_recovery: np.int32 = conditions["prob_recovery"]
    if val == SUSCEPTIBLE and CONTAMINATED in neighbours_health:
        if np.random.rand() < prob_contamination:
            return CONTAMINATED
        else:
            return val
    elif val == CONTAMINATED:
        if np.random.rand() < prob_recovery:
            return RECOVERED
        else:
            return val
    else:
        return val

def generate_epidemic_process(grid: npt.NDArray[np.int8], conditions: dict, number_of_iterations: np.int8 = 10, gif: bool = False) -> npt.NDArray[np.int8]:  
    neighbours: npt.NDArray[np.int16] = get_neighbours(grid.size)
    frames: list = []
    for _ in np.arange(number_of_iterations):
        if gif:
            frames.append(draw_grid(grid))
        grid = np.array([spread(val=val, neighbours_health=grid[neighbours[idx]], conditions=conditions) for idx, val in enumerate(grid)])
        
    create_gif(frames)


In [ ]:
GRID_SIZE: np.int32 = 9

grid: npt.NDArray[np.int8] = np.random.choice([0], size=(100))
grid[5] = 1
generate_epidemic_process(grid=grid, conditions=conditions[0], number_of_iterations=100, gif=True)
